Alumno: Paez Sheridan, Pablo Santiago
Mail: pablopaezsheridan@outlook.com.ar
Ingeniero en Sistemas de Informacion

<h1>Movies</h1> 
Paez Sheridan, Pablo Santiago
Fecha: 15/07/1997
<h2>Introducción</h2>

 En este proyecto se desarrolla en Python un análisis básico de datos sobre películas de cine
 de IMDB. El set de datos que vamos a usar inicialmente se encuentra en la siguiente
 página:
 https://www.kaggle.com/datasets/carolzhangdc/imdb-5000-movie-dataset

 En ella puede encontrarse información más detallada, así como una descripción precisa de
 cada columna.

Todo: Intro

 Se plantean los apartados iniciales para su resolución sin usar la libreria pandas. Esto se
 propone así para forzar a practicar con las técnicas, herramientas y conceptos básicos de
 Python. Más adelante, hay otros apartados propuestos para su resolución con esta librería y
 otras.

Todo: Autoevaluacion


In [259]:
# Constantes globales:

MOVIES_DATA = './movie_metadata.csv'

In [317]:
# Librerías:

import csv
import pandas as pd
import math
from collections import defaultdict

<h2>Parte A. Ejercicios básicos sin usar pandas </h2>

<h3> A.1. Exploración inicial básica del archivo de datos</h3>

In [261]:
def load_full_data(data_file_path):
    with open('movie_metadata.csv', mode='r') as file:
        reader = csv.reader(file)
        headers = next(reader)
        data = [row for row in reader]
    return headers, data

In [262]:
# Test de funcionamiento
full_header, full_list_data = load_full_data(MOVIES_DATA)
print(full_header)
print()
print(full_list_data[0:5])

['color', 'director_name', 'num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name', 'movie_title', 'num_voted_users', 'cast_total_facebook_likes', 'actor_3_name', 'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'language', 'country', 'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', 'aspect_ratio', 'movie_facebook_likes']

[['Color', 'James Cameron', '723', '178', '0', '855', 'Joel David Moore', '1000', '760505847', 'Action|Adventure|Fantasy|Sci-Fi', 'CCH Pounder', 'Avatar\xa0', '886204', '4834', 'Wes Studi', '0', 'avatar|future|marine|native|paraplegic', 'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1', '3054', 'English', 'USA', 'PG-13', '237000000', '2009', '936', '7.9', '1.78', '33000'], ['Color', 'Gore Verbinski', '302', '169', '563', '1000', 'Orlando Bloom', '40000', '309404152', 'Action|Adventure|F

Mostrar las posiciones de los identificadores de los campos, dados en la línea cabecera

In [263]:
list_of_enumerated_headers = [(i,header)for i,header in enumerate(full_header)]

In [264]:
# Test de funcionamiento
print(list_of_enumerated_headers)

[(0, 'color'), (1, 'director_name'), (2, 'num_critic_for_reviews'), (3, 'duration'), (4, 'director_facebook_likes'), (5, 'actor_3_facebook_likes'), (6, 'actor_2_name'), (7, 'actor_1_facebook_likes'), (8, 'gross'), (9, 'genres'), (10, 'actor_1_name'), (11, 'movie_title'), (12, 'num_voted_users'), (13, 'cast_total_facebook_likes'), (14, 'actor_3_name'), (15, 'facenumber_in_poster'), (16, 'plot_keywords'), (17, 'movie_imdb_link'), (18, 'num_user_for_reviews'), (19, 'language'), (20, 'country'), (21, 'content_rating'), (22, 'budget'), (23, 'title_year'), (24, 'actor_2_facebook_likes'), (25, 'imdb_score'), (26, 'aspect_ratio'), (27, 'movie_facebook_likes')]


<h3>A.2. Campos principales de una película</h3>

Los campos (columnas) del archivo son demasiados. No nos interesan todos ellos. Dada
una lista con todos los campos, se pide extraer otra lista sólo con los campos con los que
vamos a trabajar en los siguientes apartados: movie_title , title_year ,
director_name , actor_1_name , language , country , color , budget ,
imdb_score y movie_imdb_link 

In [265]:
def main_data_from_item(data:list[str]) -> list[str]:
    selected_columns = ["movie_title", "title_year", "director_name", "actor_1_name", "language", "country", "budget", "imdb_score", "movie_imdb_link"]
    selected_columns_indexes = [full_header.index(d) for c in selected_columns for d in full_header if c in d]
    return [data[si] for si in selected_columns_indexes]    

In [266]:
# Test de funcionamiento
print(main_data_from_item(full_header))
print()
datos_avatar_2009 = main_data_from_item(full_list_data[0])
print(datos_avatar_2009)
print()
datos_star_wars_7 = main_data_from_item(full_list_data[4])
print(datos_star_wars_7)

['movie_title', 'title_year', 'director_name', 'actor_1_name', 'language', 'country', 'budget', 'imdb_score', 'movie_imdb_link']

['Avatar\xa0', '2009', 'James Cameron', 'CCH Pounder', 'English', 'USA', '237000000', '7.9', 'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1']

['Star Wars: Episode VII - The Force Awakens\xa0            ', '', 'Doug Walker', 'Doug Walker', '', '', '', '7.1', 'http://www.imdb.com/title/tt5289954/?ref_=fn_tt_tt_1']


<h3>A.3. Algunos ajustes en los campos</h3>

Observa la anomalía en el string del título de la película. También, queremos tratar algunos
campos como numéricos, ya sea enteros (el año y el presupuesto) o reales (la valoración).
(Algunos campos numéricos enteros están vacíos en el archivo de datos; para ellos, el valor
imputado será -1 .) Además, en las urls de las películas no necesitaremos el fragmento
final, iniciado con ?ref_ 

In [267]:
def datatypes_arranged(data : list[str])->list[str]:
    data_treated = []
    numeric_indexes = [1,6,7]
    for index,d in enumerate(data):
        try:
            if d == "" and index in numeric_indexes:
                data_treated.append(-1)
            elif (index in numeric_indexes):
                data_treated.append(int(d))
            elif (index == 8):
                data_treated.append(d.split('?ref_')[0])
            else:
                data_treated.append(d)
        except:
            data_treated.append(d)
    return data_treated

In [268]:
# Test de funcionamiento
print(datatypes_arranged(datos_avatar_2009))
print(datatypes_arranged(datos_star_wars_7))

['Avatar\xa0', 2009, 'James Cameron', 'CCH Pounder', 'English', 'USA', 237000000, '7.9', 'http://www.imdb.com/title/tt0499549/']
['Star Wars: Episode VII - The Force Awakens\xa0            ', -1, 'Doug Walker', 'Doug Walker', '', '', -1, '7.1', 'http://www.imdb.com/title/tt5289954/']


<h3> A.4. Recuperación de alguna información básica</h3>

<p> Diseña funciones para recuperar la siguiente información: </p>
<ul>
    <li> El conjunto de valores posibles del campo `Color`.</li>
    <li> Los títulos de película de nuestro archivo (limpios de caracteres extraños), junto con la cantidad de calificadores (un entero), que contengan la subcadena "Victor".</li>        
<ul>

In [269]:
def column_values(column_index: int)-> set:
    values = set()
    column_data = [row[column_index] for row in full_list_data[1:]] 
    for d in column_data:
        values.add(d)
    return values

def film_ocurrences(search_parameter:str) -> tuple[str,int]:
    films = [()]
    for row in full_list_data[1:]:
        ocurrences = 0
        if(search_parameter in row[11]):
            ocurrences = sum([1 for f in full_list_data[1:] if row[11] in f[11]])
            films.append((row[11], ocurrences))
    return films

colores = column_values(0)
pelis_victor = film_ocurrences("Victor")

In [270]:
# Test de funcionamiento
print(colores)
print(pelis_victor)

{'', 'Color', ' Black and White'}
[(), ('Victor Frankenstein\xa0', 3), ('Victor Frankenstein\xa0', 3), ('The Young Victoria\xa0', 1), ('Victor Frankenstein\xa0', 3), ('Raising Victor Vargas\xa0', 1)]


<h2> B Datos en un diccionario [2 puntos]</h2>
<h3>B.1. Carga únicamente de datos que vamos a usar</h3>
<p> En lugar de cargar todos los datos del archivo, luego generar otro archivo con los datos que
 nos interesan y luego ajustarlos, es mejor diseñar una función que recorre el archivo de
 datos y, de cada línea del archivo, únicamente retiene la parte que nos interesa, cargando la
 cabecera por un lado y, por otro, la lista de datos de interés, ajustados como ya hemos
 dicho.
 Cargaremos esta información en un diccionario, cuyas claves serán pares (título, año)
 y cuyos valores se recogen en una lista con el resto de los campos.
 Además, los datos de cada película tras la clave (título, año) están repetidas en el
 archivo de datos con todos los datos idénticos; y lo que es peor: alguna que está repetida</p>

In [300]:
def load_main_data(data_file_path:str) -> tuple[tuple[str,int], list]:
    data_frame = pd.read_csv(data_file_path)
    data_frame = data_frame.fillna(-1)
    dict_from_df = {
    (row['movie_title'], row['title_year']): row.drop(['movie_title', 'title_year']).tolist()
    for _, row in data_frame.iterrows()}
    return dict_from_df.keys(), dict_from_df

In [301]:
# Test de funcionamiento
main_header, main_dict_data= load_main_data(MOVIES_DATA)
print(main_header)
print()
for title_year, pieces in list(main_dict_data.items())[:5]:
    print(title_year, " -> ", pieces)

dict_keys([('Avatar\xa0', 2009.0), ("Pirates of the Caribbean: At World's End\xa0", 2007.0), ('Spectre\xa0', 2015.0), ('The Dark Knight Rises\xa0', 2012.0), ('Star Wars: Episode VII - The Force Awakens\xa0            ', -1.0), ('John Carter\xa0', 2012.0), ('Spider-Man 3\xa0', 2007.0), ('Tangled\xa0', 2010.0), ('Avengers: Age of Ultron\xa0', 2015.0), ('Harry Potter and the Half-Blood Prince\xa0', 2009.0), ('Batman v Superman: Dawn of Justice\xa0', 2016.0), ('Superman Returns\xa0', 2006.0), ('Quantum of Solace\xa0', 2008.0), ("Pirates of the Caribbean: Dead Man's Chest\xa0", 2006.0), ('The Lone Ranger\xa0', 2013.0), ('Man of Steel\xa0', 2013.0), ('The Chronicles of Narnia: Prince Caspian\xa0', 2008.0), ('The Avengers\xa0', 2012.0), ('Pirates of the Caribbean: On Stranger Tides\xa0', 2011.0), ('Men in Black 3\xa0', 2012.0), ('The Hobbit: The Battle of the Five Armies\xa0', 2014.0), ('The Amazing Spider-Man\xa0', 2012.0), ('Robin Hood\xa0', 2010.0), ('The Hobbit: The Desolation of Smaug\xa

Y ahora podemos suprimir la variable full_list_data

In [273]:
del full_list_data

In [274]:

# Test de funcionamiento
try:
    print(full_list_data)
except:
    print('La variable full_list_data está suprimida correctamente')

La variable full_list_data está suprimida correctamente


<h3>B.2. Recuperación de alguna información</h3>
<p>Diseña funciones para averiguar la siguiente información:</p>
<ul>
    <li> ¿Qué títulos de películas han sido dirigidas por "James Cameron" (o por el director que
 se desee)? </li>
</ul>

In [275]:
def movies_anno_for_director(data: dict[tuple,list], director_name:str) -> list[tuple[str,int]]:
    return [key for key,film in data.items() if film[1] == director_name]
    

In [276]:
# Test de funcionamiento
movies_anno_for_director(main_dict_data, "James Cameron")

[('Avatar\xa0', 2009.0),
 ('Titanic\xa0', 1997.0),
 ('Terminator 2: Judgment Day\xa0', 1991.0),
 ('True Lies\xa0', 1994.0),
 ('The Abyss\xa0', 1989.0),
 ('Aliens\xa0', 1986.0),
 ('The Terminator\xa0', 1984.0)]

<li> ¿Qué directores han dirigido el número máximo de películas?</li>

In [277]:
def directors_max_movies(data: dict[tuple,list]) -> tuple[list[str],int]:
    first_film_data = data[next(iter(data))]
    directors_and_film_numbers = [[first_film_data[1],0]]
    for f in data.items():
        for d in directors_and_film_numbers:
            if f[1][1] == d[0]:
                d[1] += 1
                break
        directors_and_film_numbers.append([f[1][1],1])
    return tuple(max(directors_and_film_numbers , key = lambda t: t[1]))
        


In [278]:
# Test de funcionamiento
print(directors_max_movies(main_dict_data))

('Steven Spielberg', 26)


<li>Para cada año de un intervalo dado de años, ¿cuántas películas se han realizado? Esta
 información debe recuperrase en un diccionario convencional (no por defecto), y luego
 debe mostrarse en una lista con los años en orden ascendente.</li>

In [279]:

def years_num_movies(data: dict[tuple,list], initial_year: int, end_year:int) -> dict[int,int]:
    number_of_films = {initial_year:0}
    for year in range(initial_year, end_year+1):
        number_of_films[year] = sum([1 for fk in data.keys() if fk[1] == float(year)])
    return number_of_films


In [280]:
# Test de funcionamiento
num_movies = years_num_movies(main_dict_data, 2000, 2015)
print(num_movies)

{2000: 169, 2001: 183, 2002: 204, 2003: 169, 2004: 207, 2005: 216, 2006: 235, 2007: 198, 2008: 223, 2009: 253, 2010: 225, 2011: 224, 2012: 214, 2013: 231, 2014: 243, 2015: 211}


Diseña también una operación que, partiendo de nuestro diccionario de 
los datos principales, almacene en un archivo los datos siguientes de cada 
película, sin la cabecera: el título, el idioma, el año, el país y el 
presupuesto. El separador será en este caso el carácter `|`. El archivo 
usado para el almacenamiento es el siguiente

``` python
 FEW_FIELDS = "algunos_campos.txt"
 ```

In [ ]:
FEW_FIELDS = "algunos_campos.txt"

def store_file(data:dict[tuple[str,int],list], file_path:str):
    with open(file_path, "w") as file:
        for r in data.items():
           line = str(r[0][0]) + "|" + str(r[1][18]) + "|" + str(r[0][1]) + "|" + str(r[1][19]) + "|" + str(r[1][21])
           file.write(line + "\n")

In [313]:
# Test de funciomaniemto
store_file(main_dict_data, FEW_FIELDS)

! dir algunos*.*
print()
with open(FEW_FIELDS) as f:
    for i in range(5):
        print(f.readline())

 Volume in drive C is Windows
 Volume Serial Number is 6566-6370

 Directory of c:\Users\pablo\OneDrive\Documentos\Master Ciencia de Datos\01 - Python\Tarea\FilmDataAnalysis\solution

06/11/2024  17:48           241.678 algunos_campos.txt
               1 File(s)        241.678 bytes
               0 Dir(s)  733.283.643.392 bytes free

Avatar |English|2009.0|USA|237000000.0

Pirates of the Caribbean: At World's End |English|2007.0|USA|300000000.0

Spectre |English|2015.0|UK|245000000.0

The Dark Knight Rises |English|2012.0|USA|250000000.0

Star Wars: Episode VII - The Force Awakens             |-1|-1.0|-1|-1.0



<h3> B.3. Un conteo sencillo con defaultdict</h3>
<p> Deseamos saber con qué directores y número de veces ha actuado cada actor como actor
 principal. Se pide realizar este conteo en un defaultdict cuyas claves serán nombres de
 los actores y cuyos valores tendrán la estructura de un defaultdict , cuyas claves serán
 los nombres de los directores y cuyos valores serán los núneros contabilizados</p>

In [336]:
def actor_directors(data:dict[tuple[str,int],list])-> dict[dict[int]]:
    actors = {}
    for actor_films in data.items():
        directors = defaultdict(int)
        for director_films in data.items():
            if director_films[1][10] == actor_films[1][10]:
                directors[director_films[1][1]]+=1
        actors[actor_films[1][10]] = directors
    return actors

In [337]:
# Test de funcionamiento
num_collaborations = actor_directors(main_dict_data)

print(type(num_collaborations))
key_a, value_a = list(num_collaborations.items())[0]
print(type(key_a), type(value_a))
key_b, value_b = list(value_a.items())[0]
print(type(key_b), type(value_b))

print()

print(num_collaborations)

<class 'dict'>
<class 'str'> <class 'collections.defaultdict'>
<class 'str'> <class 'int'>

{'CCH Pounder': defaultdict(<class 'int'>, {'James Cameron': 1, 'Peter Hyams': 1, 'Fred Dekker': 1, 'Ernest R. Dickerson': 1}), 'Johnny Depp': defaultdict(<class 'int'>, {'Gore Verbinski': 5, 'Rob Marshall': 2, 'Tim Burton': 6, 'James Bobin': 1, 'Michael Mann': 1, 'Florian Henckel von Donnersmarck': 1, 'Wally Pfister': 1, 'David Koepp': 2, 'Scott Cooper': 1, 'Albert Hughes': 1, 'Roman Polanski': 1, 'Wes Craven': 1, 'Mike Newell': 1, 'Rand Ravich': 1, 'Ted Demme': 1, 'Robert Rodriguez': 1, 'Marc Forster': 1, 'Jeremy Leven': 1, 'Laurence Dunmore': 1, 'Terry Gilliam': 1, 'Lasse Hallström': 1, 'Oliver Stone': 1, 'Rachel Talalay': 1, 'Kevin Smith': 2}), 'Christoph Waltz': defaultdict(<class 'int'>, {'Sam Mendes': 1, 'David Yates': 1, 'Michel Gondry': 1, 'Tim Burton': 1}), 'Tom Hardy': defaultdict(<class 'int'>, {'Christopher Nolan': 1, 'George Miller': 1, 'McG': 1, 'Stuart Baird': 1, 'Daniel Espinosa

<h3>B.4. Print seleccionado</h3>
<p> Demasiada información. Deseamos imprimir únicamente, para cada actor, las
 colaboraciones que superen un mínimo, dato entrada. Si un actor no tiene ninguna
 colaboración que supere dicho mínimo, lógicamente no debe mostrarse.</p>

In [364]:
num_collaborations = actor_directors(main_dict_data)
for actor in num_collaborations.items():
    keys_to_remove = [director for director in actor[1] if actor[1][director] < 5]
    for key in keys_to_remove:
        del actor[1][key]

for actor, directors in num_collaborations.items():
    if len(directors) > 0:
        print(actor + ' -> ' + str([(dk,dv) for dk, dv in directors.items()]) )



Johnny Depp -> [('Gore Verbinski', 5), ('Tim Burton', 6)]
Leonardo DiCaprio -> [('Martin Scorsese', 5)]
Robert De Niro -> [('Martin Scorsese', 7)]
Bill Murray -> [('Wes Anderson', 5)]
Clint Eastwood -> [('Clint Eastwood', 10)]
Woody Allen -> [('Woody Allen', 10)]
